In [2]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kennd/Downloads/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [4]:
## Import all modules needed
## lem is lemmatizing the words
count = 0
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():
        count += 1 
        if count % 10000 == 0:
            print(count)        
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    count = 0
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(min_df=3,max_df=20,stop_words=stops,ngram_range=(1,1))
tfvectorizer = TfidfVectorizer(stop_words=stops,max_df=20, min_df=3,ngram_range=(1,1))

import pandas as pd
import pickle
with open("C:/Users/kennd/Documents/Github/mcnulty_yelp/data/review_business_df.pkl", 'rb') as picklefile: 
    review_business_df = pickle.load(picklefile)

In [5]:
review_business_df

,text,stars_x,date,business_id,user_id,review_id,name,city,open,review_count,stars_y,state,latitude,longitude
0,"Mr Hoagie is an institution. Walking in, it do...",4,2012-08-01,5UmKMjUEUNdYWqANhGckJw,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,Mr Hoagie,Dravosburg,True,4,4.5,PA,40.354327,-79.900706
1,Excellent food. Superb customer service. I mis...,5,2014-02-13,5UmKMjUEUNdYWqANhGckJw,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Mr Hoagie,Dravosburg,True,4,4.5,PA,40.354327,-79.900706
2,Yes this place is a little out dated and not o...,5,2015-10-31,5UmKMjUEUNdYWqANhGckJw,auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Mr Hoagie,Dravosburg,True,4,4.5,PA,40.354327,-79.900706
3,This place was DELICIOUS!! My parents saw a r...,5,2012-12-01,mVHrayjG3uZ_RLHkLj-AMg,LWbYpcangjBMm4KPxZGOKg,6w6gMZ3iBLGcUM4RBIuifQ,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247
4,Can't miss stop for the best Fish Sandwich in ...,5,2013-03-15,mVHrayjG3uZ_RLHkLj-AMg,m1FpV3EAeggaAdfPx0hBRQ,jVVv_DA5mCDB6mediuwHAw,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247
5,This place should have a lot more reviews - bu...,5,2013-03-30,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,3Es8GsjkssusYgeU6_ZVpQ,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247
6,This place was very good. I found out about Em...,4,2013-10-20,mVHrayjG3uZ_RLHkLj-AMg,uK8tzraOp4M5u3uYrqIBXg,KAkcn7oQP1xX8KsZ-XmktA,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247
7,"Old school.....traditional ""mom 'n pop"" qualit...",5,2013-11-07,mVHrayjG3uZ_RLHkLj-AMg,6wvlM5L4_EroGXbnb_92xQ,BZNJkkP0bXnwQ2-sCqat2Q,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247
8,Seen this restaurant on 25 best places in Pitt...,5,2014-03-22,mVHrayjG3uZ_RLHkLj-AMg,345nDw0oC-jOcglqxmzweQ,VDTIbR3G5_IPkpXbo2MutA,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247
9,"Like a lot of recent patrons, I heard about Em...",5,2014-05-01,mVHrayjG3uZ_RLHkLj-AMg,r1Uk-h_bddXb2ScCetoMPA,PY6nIMMLP2pqARlbIOJuRA,Emil's Lounge,rankin,True,20,5.0,PA,40.413464,-79.880247


In [6]:
short_df = review_business_df.iloc[:,0:2]

In [9]:
short_df['spell'] = short_df['text'].apply(lambda x: x.lower())

In [10]:
short_df['s_spell_lem'] = short_df['spell'].apply(lem)

KeyboardInterrupt: 

In [11]:
new_df = short_df[['spell','stars_x']]

In [14]:
corp = list(new_df.spell)

In [15]:
feature = vectorizer.fit_transform(corp)

In [18]:
feature.A

MemoryError: 

In [16]:
featuredf = pd.DataFrame(feature.A, columns=vectorizer.get_feature_names())

MemoryError: 

In [ ]:
new_df = pd.concat((new_df,featuredf),axis=1)

# Model testing

In [ ]:
##Split into train and test at 75/25
from sklearn.cross_validation import train_test_split 
train, test = train_test_split(new_df,test_size = 0.25)

In [ ]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,nb_Y_pred,labels=['1','2','3','4','5']))

## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcl_Y_pred,labels=['1','2','3','4','5']))

## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=1).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcrbf_Y_pred,labels=['1','2','3','4','5']))

## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,log_Y_pred,labels=['1','2','3','4','5']))

## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,dt_Y_pred,labels=['1','2','3','4','5']))

## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,rf_Y_pred,labels=['1','2','3','4','5']))